<a href="https://colab.research.google.com/github/SWLee1212/TensorFlow-in-Practice/blob/master/NLP_in_TensorFlow_2_week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()

1.14.0


In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
!pip install -q tensorflow-datasets

In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [5]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=0.1.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    urls=['http://ai.stanford.edu/~amaas/data/sentiment/'],
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word V

In [0]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())  




In [12]:
sentence = tf.constant("This movie could be an all-time hit!")
label = tf.constant(1)
# we can print tensors directly without creating 
# tf.Session() objects, thanks to eager execution
print(sentence)
print(label)

tf.Tensor(b'This movie could be an all-time hit!', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int32)


In [0]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
# hyperparameters
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length,)

In [24]:
print(sequences[0])
print(len(padded[0]))

[59, 11, 216, 12, 20, 22, 284, 101, 65, 5411, 93, 73, 1, 19, 164, 10, 14, 52, 138, 24, 87, 5, 2, 490, 115, 1315, 42, 2896, 422, 5, 65, 44, 7, 118, 19, 87, 164, 484, 51, 82, 136, 11, 196, 2, 20, 14, 1884, 9, 103, 2846, 5, 2368, 1976, 17, 1, 8, 8, 103, 24, 1, 6034, 1223, 24, 1, 654, 1963, 106, 36, 1, 145, 3149, 17, 1, 1877, 1223, 357, 3, 34, 518, 39, 357, 64, 8, 8, 1223, 4592, 2122, 3, 8986, 1496, 45, 285, 434, 12, 99, 29, 33, 1725, 3, 34, 99, 43, 29, 80, 156, 19, 11, 174, 105, 37, 1, 1, 415, 55, 24, 269, 6, 29, 654, 4, 54, 873, 234, 3, 11, 455, 90, 16, 10, 5682, 6665, 7, 6588, 3, 624, 19, 56, 14, 2763, 3, 4, 2368, 2, 2249, 5, 2, 108, 463, 627, 1, 22, 131, 1, 2, 115, 14, 54, 737, 3, 19, 37, 51, 11, 2436, 98, 26, 4413, 3, 5199, 6, 5618, 40, 2521, 243, 1092, 5619, 1036, 6, 728, 98, 2461, 296, 11, 105, 2, 106, 925, 33, 254, 336, 64, 856, 28, 78, 624, 3, 1316, 19, 34, 2498, 10, 5620, 2, 1, 5, 2, 915, 1223, 9, 13, 565, 1287, 9, 715, 3, 56, 2392, 22, 7141, 17, 2, 899, 11, 428, 1636, 32, 40, 16

In [26]:
print(testing_sequences[0])

[59, 402, 298, 2, 18, 164, 458, 207, 3, 51, 11, 182, 6, 136, 45, 10, 7, 2, 67, 153, 13, 93, 12, 18, 315, 4273, 14, 2, 1, 1, 1061, 234, 439, 163, 19, 13, 24, 62, 149, 719, 2183, 783, 1, 1, 3, 50, 5, 135, 1082, 1458, 32, 207, 9, 2, 18, 1, 7, 416, 54, 1031, 402, 111, 4, 169, 2669, 27, 93, 3, 78, 145, 62, 152, 12, 7, 239, 5, 189, 18, 565, 406, 68, 52, 293, 9, 209, 101, 3, 84, 9, 3308, 100, 11, 2154, 92, 5, 23, 31, 216, 1, 1, 2, 18, 3, 242, 198, 149, 108, 64, 141, 16, 35, 473, 31, 319, 2168, 136, 7, 23, 130, 107, 46, 16, 2, 159, 504, 596, 46, 38, 1, 95, 1001, 69, 410, 2, 52, 157, 1, 77, 119]


In [27]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b this is the most depressing film i have ever seen i first saw it as a child and even thinking about it now really <OOV> me i know it was set in a time when life was hard and i know these people were poor and the <OOV> were vital yes i get all that what i find hard to take is i can't remember one single light moment in the entire film maybe it was true to life i don't know i'm quite sure the acting was top notch and the direction and quality of filming etc etc was wonderful and i know that every film can't have a happy ending but as a family film it is
b"This is the most depressing film I have ever seen. I first saw it as a child and even thinking about it now really upsets me. I know it was set in a time when life was hard and I know these people were poor and the crops were vital. Yes, I get all that. What I find hard to take is I can't remember one single light moment in the entire film. Maybe it was true to life, I don't know. I'm quite sure the acting was top notch and the direct

In [34]:
print(reverse_word_index.get('?'))

None


In [38]:
def decode_review_test(text):
    return ' '.join([reverse_word_index.get(i) for i in text])

print(decode_review_test(padded[1]))
print(decode_review(padded[1]))

print(decode_review_test(padded[1]) == decode_review(padded[1]) )

b this is the most depressing film i have ever seen i first saw it as a child and even thinking about it now really <OOV> me i know it was set in a time when life was hard and i know these people were poor and the <OOV> were vital yes i get all that what i find hard to take is i can't remember one single light moment in the entire film maybe it was true to life i don't know i'm quite sure the acting was top notch and the direction and quality of filming etc etc was wonderful and i know that every film can't have a happy ending but as a family film it is
b this is the most depressing film i have ever seen i first saw it as a child and even thinking about it now really <OOV> me i know it was set in a time when life was hard and i know these people were poor and the <OOV> were vital yes i get all that what i find hard to take is i can't remember one single light moment in the entire film maybe it was true to life i don't know i'm quite sure the acting was top notch and the direction and q

In [39]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [41]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
25000/25000 [==============================] - 5s 215us/sample - loss: 0.5164 - acc: 0.7176 - val_loss: 0.3575 - val_acc: 0.8419
Epoch 2/10
25000/25000 [==============================] - 5s 189us/sample - loss: 0.2486 - acc: 0.9053 - val_loss: 0.3663 - val_acc: 0.8396
Epoch 3/10
25000/25000 [==============================] - 5s 188us/sample - loss: 0.0978 - acc: 0.9750 - val_loss: 0.4660 - val_acc: 0.8205
Epoch 4/10
25000/25000 [==============================] - 5s 193us/sample - loss: 0.0280 - acc: 0.9960 - val_loss: 0.5173 - val_acc: 0.8266
Epoch 5/10
25000/25000 [==============================] - 5s 188us/sample - loss: 0.0110 - acc: 0.9986 - val_loss: 0.5672 - val_acc: 0.8282
Epoch 6/10
25000/25000 [==============================] - 5s 190us/sample - loss: 0.0053 - acc: 0.9993 - val_loss: 0.6168 - val_acc: 0.8257
Epoch 7/10
25000/25000 [==============================] - 5s 187us/sample - loss: 0.0015 - acc: 0.9999 - val_loss: 0.6671 - val_acc: 0.8252
Epoch 8/10
25000/250

In [42]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [43]:
len(model.layers)

4

In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [48]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1430], [968], [4], [1537], [1537], [4738], [], [790], [2015], [11], [2922], [2189], [], [790], [2015], [11], [579], [], [11], [579], [], [4], [1783], [4], [4508], [11], [2922], [1277], [], [], [2015], [1005], [2922], [968], [579], [790], []]


In [50]:
testing_padded.shape

(25000, 120)

In [52]:
np.array(sequence)

array([list([11]), list([]), list([1430]), list([968]), list([4]),
       list([1537]), list([1537]), list([4738]), list([]), list([790]),
       list([2015]), list([11]), list([2922]), list([2189]), list([]),
       list([790]), list([2015]), list([11]), list([579]), list([]),
       list([11]), list([579]), list([]), list([4]), list([1783]),
       list([4]), list([4508]), list([11]), list([2922]), list([1277]),
       list([]), list([]), list([2015]), list([1005]), list([2922]),
       list([968]), list([579]), list([790]), list([])], dtype=object)